In [2]:
import numpy as np
import pandas as pd
from ideas import intersection_over_union
from ideas import al
from ideas import aw

In [5]:
box_columns = ['Xmin','Ymin','Xmax','Ymax']

def weighted_averaging(data, item_id, np_data, avg):
    ix = data[data['itemId'] == item_id].index
    vi = np_data[ix]
    l = np.fmax(al(vi), np.ones(len(vi), dtype=int))
    w = np.fmax(aw(vi), np.ones(len(vi), dtype=int))
    sabx = (l*w)**2
    sb = sum(1/sabx)
    for y in range(0,4):
        avg.loc[item_id][y] = sum(vi[:,y]/sabx)/sb

In [3]:
t_a = pd.read_csv("train_answers.csv")
t_d = pd.read_csv("train_data.csv")
test_data = pd.read_csv('test_data.csv')
users_rating = pd.read_csv("users_rating.csv")
users_rating = users_rating.set_index('userId')
users_adj = pd.read_csv("users_adj.csv")
users_adj = users_adj.set_index('userId')
max_result = pd.read_csv('submit-max.csv')
max_result = max_result.set_index('itemId')

In [4]:
ids = list(set(test_data['itemId']))
bx = test_data.loc[: 'itemId'][box_columns]
abx = np.array(bx)

NameError: name 'box_columns' is not defined

In [377]:
answer = pd.DataFrame(np.zeros((len(ids), 4), dtype=int), index=ids, columns=box_columns)
for i in ids:
    weighted_averaging(test_data, i, abx, answer)

In [151]:
iid = 33980
uids = test_data[test_data['itemId'] == iid]['userId']
uid = users_rating.loc[uids].idxmax()['iou']
ti = test_data.loc[test_data['itemId'] == iid].loc[test_data['userId'] == uid][box_columns]
# for i in box_columns:
#     answer.loc[ids[0]][i] = ti[i]

In [12]:
answer = pd.DataFrame(np.zeros((len(ids), 5), dtype=int), index=ids, columns=box_columns + ['itemId'])
for i in ids:    
    uids = test_data[test_data['itemId'] == i]['userId']
    uid = users_rating.loc[uids].idxmax()['iou']
    ti = test_data.loc[test_data['itemId'] == i].loc[test_data['userId'] == uid][box_columns].max()
    uajd = users_adj.loc[uid]
    l = ti['Xmax'] - ti['Xmin']
    w = ti['Ymax'] - ti['Ymin']
    for y in box_columns:        
        t = l if "X" in y else w
        c = 1 if "min" in y else -1
        answer.loc[i][y] = answer.loc[i][y] + c * t * uajd[y + 'DifPer']
    answer.loc[i]['itemId'] = i  

/home/vasste/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  after removing the cwd from sys.path.


       Xmin  Ymin  Xmax  Ymax  itemId
4099      3    11    60     6    4099
8199     -2     8    19   -33    8199
8200   -104    26    44   -16    8200
24587   -25    -5    30    -8   24587
8204     20     2     0    -8    8204
12300   -78   -58   116    63   12300
18       -6    -5    56    38      18
19       -1    -4     8    26      19
6163     33    41    56    72    6163
26645   -34    14    36   -39   26645
18454     0     0     0     0   18454
28694   -39   -25    58    27   28694
30745   -22   -26   -33   -60   30745
2075     48    53   -14    76    2075
2081      6   -38   -15    19    2081
33      -55   -12   -40    -8      33
18470  -140   -36  -220   -82   18470
14376    -5     9     8    -5   14376
10280   -21    -5    -5   -26   10280
32812     4     4     4     0   32812
22572    19    16     8   -47   22572
24633    51    44    84   -13   24633
30777   -15   -25   -22   -58   30777
62      -92    -4   -66    -3      62
24639    74    15   -29     0   24639
16449    25 

In [9]:
data = answer.merge(max_result, on=['itemId'])
data["iou"] = data[['Xmin','Ymin', 'Xmax', 'Ymax', 'Xmin_true',\
      'Ymin_true', 'Xmax_true','Ymax_true']].apply(intersection_over_union, axis=1)
print(data["iou"].mean()) 

       Xmin  Ymin  Xmax  Ymax  itemId
4099     -3   -11   -60    -6    4099
8199      2    -8   -19    33    8199
8200    104   -26   -44    16    8200
24587    25     5   -30     8   24587
8204    -20    -2     0     8    8204
12300    78    58  -116   -63   12300
18        6     5   -56   -38      18
19        1     4    -8   -26      19
6163    -33   -41   -56   -72    6163
26645    34   -14   -36    39   26645
18454     0     0     0     0   18454
28694    39    25   -58   -27   28694
30745    22    26    33    60   30745
2075    -48   -53    14   -76    2075
2081     -6    38    15   -19    2081
33       55    12    40     8      33
18470   140    36   220    82   18470
14376     5    -9    -8     5   14376
10280    21     5     5    26   10280
32812    -4    -4    -4     0   32812
22572   -19   -16    -8    47   22572
24633   -51   -44   -84    13   24633
30777    15    25    22    58   30777
62       92     4    66     3      62
24639   -74   -15    29     0   24639
16449   -25 

In [11]:
answer.to_csv('submit14.csv', header=False)